In [4]:
library(nichenetr)
library(Seurat)
library(SeuratObject)
library(tidyverse)
library(dplyr)
library(circlize)
library(networkD3)

### Notations for Input Data in Spacia

- **Bag (𝑩𝒊)**: Represents a receiving cell with nearby sending cells (instances).
- **𝒚𝒊**: Observed binary label for bag 𝒊 (activation status of the receiving gene/pathway).
- **𝒏**: Total number of bags (receiving cells) observed.
- **𝒎𝒊**: Number of sending cells in bag 𝒊.

#### Expression Component
- **𝑿𝒊𝒆**: Design matrix for gene/pathway expression in bag 𝒊, of size 𝒎𝒊 × (𝒆 + 𝟏).
- **𝒙𝒊𝒋𝒆**: Row vector for sending cell 𝒋 in bag 𝒊, of length 𝒆 + 𝟏.
  - **𝒙𝒊𝒋𝒆 = (𝟏, 𝒙𝒊𝒋𝟏, ..., 𝒙𝒊𝒋𝒆)**: Includes an intercept (1) and expression features.

#### Distance Component
- **𝑿𝒊𝒅**: Design matrix for cell-to-cell distances in bag 𝒊, of size 𝒎𝒊 × 𝟐.
- **𝒙𝒊𝒋𝒅**: Row vector for the distance of sending cell 𝒋 to the receiving cell in bag 𝒊.
  - **𝒙𝒊𝒋𝒅 = (𝟏, 𝒅𝒊𝒋)**: Includes an intercept (1) and distance (𝒅𝒊𝒋).

#### Primary Instances
- **𝐁𝒊∗**: Set of primary instances within bag 𝒊 (sending cells that truly influence the receiving cell).
- **𝛅𝒊𝒋**: Latent indicator variable for primary instance status of sending cell 𝒋 in bag 𝒊.
  - **𝛅𝒊𝒋 = 1** if sending cell 𝒋 is a primary instance of bag 𝒊.
  - **𝛅𝒊𝒋 = 0** otherwise.


In [3]:
lr_network = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/lr_network_mouse_21122021.rds")
ligand_target_matrix = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/ligand_target_matrix_nsga2r_final_mouse.rds")
weighted_networks = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/weighted_networks_nsga2r_final_mouse.rds")
ligand_tf_matrix = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/ligand_tf_matrix_nsga2r_final_mouse.rds")
sig_network = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/signaling_network_mouse_21122021.rds")
gr_network = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/gr_network_mouse_21122021.rds")

weighted_networks_lr = weighted_networks$lr_sig %>% inner_join(lr_network, by = c("from","to"))
weighted_networks_lr=weighted_networks_lr[,-c(4,5)]

In [4]:
pvat_8weeks_hf_male_expr = read.csv("../exported_data/pvat_8weeks_hf_male_expr.csv", row.names=1)
pvat_8weeks_hf_male_metadata = read.csv("../exported_data/pvat_8weeks_hf_male_metadata.csv", row.names=1)

In [5]:
head(pvat_8weeks_hf_male_expr)

,X1700066B19Rik,X3110082J24Rik,X4930404H24Rik,X4930455H04Rik,X4933400A11Rik,X6330411D24Rik,A1cf,A2m,A2ml1,A3galt2,⋯,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,hist1h2ail2,mrpl11,mrpl24,mrpl9,rnf141
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACCCACAACTTCTT-1-3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,1,1,0,0,0,0,0
AAACCCAGTCCCGTGA-1-3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACCCATCGACACCG-1-3,0,0,0,0,0,0,0,0,0,0,⋯,0,1,0,0,0,0,1,0,0,0
AAACCCATCTCAACGA-1-3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGAAAGTAAGGGA-1-3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,1,0,0,1,0,0
AAACGAACACATATCG-1-3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,1,0,0,0,0,0


In [7]:
head(pvat_8weeks_hf_male_metadata)

,sample_id,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,⋯,systolic_bp_7W,MAP_9W,systolic_bp_9W,MAP_16W,systolic_bp_16W,MAP_24W,systolic_bp_24W,doublet,celltype,celltype_broad
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>
AAACCCACAACTTCTT-1-3,P_HF_8W_M_01,1215,7.103322,1902,7.551187,10.147213,16,2.8332133,0.84121977,1,⋯,174.33,NA,NA,NA,NA,NA,NA,False,Fibroblasts_Bmper+_Nrxn1+,Fibroblasts
AAACCCAGTCCCGTGA-1-3,P_HF_8W_M_01,1754,7.470224,3375,8.124447,9.037037,1,0.6931472,0.02962963,18,⋯,174.33,NA,NA,NA,NA,NA,NA,False,Adipocytes_Brown,Adipocytes
AAACCCATCGACACCG-1-3,P_HF_8W_M_01,2440,7.800163,5511,8.614683,9.653420,4,1.6094379,0.07258211,25,⋯,174.33,NA,NA,NA,NA,NA,NA,False,Adipocytes_Brown,Adipocytes
AAACCCATCTCAACGA-1-3,P_HF_8W_M_01,1001,6.909753,1480,7.300473,9.256757,14,2.7080502,0.94594595,5,⋯,174.33,NA,NA,NA,NA,NA,NA,False,ECs_Cap,Endothelial_Cells
AAACGAAAGTAAGGGA-1-3,P_HF_8W_M_01,1026,6.934397,1439,7.272398,8.269632,6,1.9459101,0.41695622,5,⋯,174.33,NA,NA,NA,NA,NA,NA,False,Adipocytes_Brown,Adipocytes
AAACGAACACATATCG-1-3,P_HF_8W_M_01,738,6.605298,1131,7.031741,12.201592,15,2.7725887,1.32625995,3,⋯,174.33,NA,NA,NA,NA,NA,NA,False,ECs_Cap,Endothelial_Cells


In [8]:
seuratObj = CreateSeuratObject(counts = t(pvat_8weeks_hf_male_expr), meta.data = pvat_8weeks_hf_male_metadata)  # counts should be a cell by gene matrix and not gene by cell

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [9]:
table(seuratObj@meta.data$celltype_broad)


       Adipocytes Endothelial_Cells       Fibroblasts      Immune_Cells 
             6429               876              1077               577 
Mesothelial_Cells    Neuronal_Cells         Pericytes              SMCs 
              193                22               426                 8 

In [10]:
cell_data <- list(seuratObj = seuratObj,
                  expression_matrix = t(pvat_8weeks_hf_male_expr),
                  transposed_expression_matrix = pvat_8weeks_hf_male_expr
                 )

In [11]:
# List nicheNet analysis components using mouse preknowledge cell-cell communication data.
nichenet_networks <- list(
  lr_network = lr_network,
  ligand_target_matrix = ligand_target_matrix,
  weighted_networks = weighted_networks,
  ligand_tf_matrix = ligand_tf_matrix,
  sig_network = sig_network,
  gr_network = gr_network,
  weighted_networks_lr = weighted_networks_lr
)

In [12]:
# Identify genes with high expression levels across cells.
highly_expressed_genes <- function(expression_matrix, cell_ids) {
    
    transposed_expression_matrix = t(expression_matrix)
    
    genes <- transposed_expression_matrix[cell_ids, ] %>%
    apply(2, function(x) {10 * (2^x - 1)}) %>%
    apply(2, function(x) {log2(mean(x) + 1)}) %>%
    .[. >= 2] %>%
    names()
  return(genes)
    
    }

In [13]:
# Generate a heatmap for active ligand-target interactions using specified ligands and targets.
ligand_target_heatmap <- function(active_ligand_target_links_df, ligand_target_matrix, best_upstream_ligands) {

    active_ligand_target_links = prepare_ligand_target_visualization(ligand_target_df = active_ligand_target_links_df, ligand_target_matrix = ligand_target_matrix, cutoff = 0.33)
    order_ligands = intersect(best_upstream_ligands, colnames(active_ligand_target_links)) %>% rev() %>% make.names()
    order_targets = active_ligand_target_links_df$target %>% unique() %>% intersect(rownames(active_ligand_target_links)) %>% make.names()
    rownames(active_ligand_target_links) = rownames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    colnames(active_ligand_target_links) = colnames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    vis_ligand_target = active_ligand_target_links[order_targets,order_ligands] %>% t()
    options(repr.plot.width=20, repr.plot.height=10)
    #p_ligand_target_network = vis_ligand_target %>% make_heatmap_ggplot("Prioritized ligands","Predicted target genes", color = "purple",legend_position = "top", x_axis_position = "top",legend_title = "Regulatory potential")  + theme(axis.text.x = element_text(face = "italic")) + scale_fill_gradient2(low = "whitesmoke",  high = "purple", breaks = c(0,0.0045,0.0090))
    #p_ligand_target_network = p_ligand_target_network + theme(axis.text.x = element_text(face = "italic", size = 20),
    #                                    axis.text.y = element_text(size = 20),
    #                                   axis.title.x = element_text(size = 20),
    #                                   axis.title.y = element_text(size = 20),
    #                                   plot.title = element_text(hjust = 0.5))

    return(vis_ligand_target)
    
}

In [14]:
# Construct and order a heatmap for top ligand-receptor interactions
ligand_receptor_heatmap <- function(lr_network, best_upstream_ligands, expressed_receptors, weighted_networks_lr) {

    lr_network_top = lr_network %>% filter(from %in% best_upstream_ligands & to %in% expressed_receptors) %>% distinct(from,to)
    best_upstream_receptors = lr_network_top %>% pull(to) %>% unique()
    lr_network_top_df_large = weighted_networks_lr %>% filter(from %in% best_upstream_ligands & to %in% best_upstream_receptors)
    lr_network_top_df = lr_network_top_df_large %>% spread("from","weight",fill = 0)
    lr_network_top_matrix = lr_network_top_df %>% select(-to) %>% as.matrix() %>% magrittr::set_rownames(lr_network_top_df$to)
    dist_receptors = dist(lr_network_top_matrix, method = "binary")
    hclust_receptors = hclust(dist_receptors, method = "ward.D2")
    order_receptors = hclust_receptors$labels[hclust_receptors$order]
    dist_ligands = dist(lr_network_top_matrix %>% t(), method = "binary")
    hclust_ligands = hclust(dist_ligands, method = "ward.D2")
    order_ligands_receptor = hclust_ligands$labels[hclust_ligands$order]
    order_receptors = order_receptors %>% intersect(rownames(lr_network_top_matrix))
    order_ligands_receptor = order_ligands_receptor %>% intersect(colnames(lr_network_top_matrix))
    vis_ligand_receptor_network = lr_network_top_matrix[order_receptors, order_ligands_receptor]
    rownames(vis_ligand_receptor_network) = order_receptors %>% make.names()
    colnames(vis_ligand_receptor_network) = order_ligands_receptor %>% make.names()
    options(repr.plot.width=20, repr.plot.height=10)
    #p_ligand_receptor_network = vis_ligand_receptor_network %>% t() %>% make_heatmap_ggplot("Ligands","Receptors", color = "mediumvioletred", x_axis_position = "top",legend_title = "Prior interaction potential")
    #p_ligand_receptor_network = p_ligand_receptor_network + theme(axis.text.x = element_text(face = "italic", size = 20), 
    #                                    axis.text.y = element_text(size = 20),
    #                                   axis.title.x = element_text(size = 20),
    #                                   axis.title.y = element_text(size = 20)) 
    return(vis_ligand_receptor_network)
    }

In [15]:
# Analyze ligand-receptor activities between specified source and target cell types using nicheNet networks and cell data.

ligand_receptor_activites <- function(source_celltype, target_celltype, cell_data, nichenet_networks) {
    
    lr_network = nichenet_networks$lr_network
    ligand_target_matrix = nichenet_networks$ligand_target_matrix
    weighted_networks = nichenet_networks$weighted_networks
    ligand_tf_matrix = nichenet_networks$ligand_tf_matrix
    sig_network = nichenet_networks$sig_network
    gr_network = nichenet_networks$gr_network
    weighted_networks_lr = nichenet_networks$weighted_networks_lr
    seuratObj = cell_data$seuratObj
    expression_matrix = cell_data$expression_matrix
    transposed_expression_matrix = cell_data$transposed_expression_matrix

    n = 500
    #number_of_target_genes = n
    
    source_cell_ids = WhichCells(seuratObj, expression = celltype_broad == source_celltype)
    target_cell_ids = WhichCells(seuratObj, expression = celltype_broad == target_celltype)
    
    source_genes = highly_expressed_genes(expression_matrix, source_cell_ids)
    target_genes = highly_expressed_genes(expression_matrix, target_cell_ids)
    #n = length(target_genes)-1
    
    Idents(seuratObj) = seuratObj$celltype
    
    background_expressed_genes = target_genes %>% .[. %in% rownames(ligand_target_matrix)]
    ligands = lr_network %>% pull(from) %>% unique()
    expressed_ligands = intersect(ligands, source_genes)
    receptors = lr_network %>% pull(to) %>% unique()
    expressed_receptors = intersect(receptors, target_genes)
    potential_ligands = lr_network %>% filter(from %in% expressed_ligands & to %in% expressed_receptors) %>% pull(from) %>% unique()
    
    ligand_activities = predict_ligand_activities(geneset = (target_genes[0:n]), background_expressed_genes = background_expressed_genes, ligand_target_matrix = ligand_target_matrix, potential_ligands = potential_ligands)
    ligand_activities = ligand_activities %>% arrange(-aupr_corrected)
    best_upstream_ligands = ligand_activities %>% arrange(-aupr_corrected) %>% pull(test_ligand)
    best_upstream_ligands = best_upstream_ligands %>% intersect(expressed_ligands)

    
    ligand_expression_tbl = tibble(
    ligand = best_upstream_ligands, 
    source = transposed_expression_matrix[source_cell_ids, best_upstream_ligands] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}))

    general_ligands = ligand_expression_tbl %>% pull(ligand)
    
    ligand_type_indication_df = tibble(
                                ligand_type = c(rep(source_celltype, times = general_ligands %>% length())),
                                ligand = c(general_ligands))


    active_ligand_target_links_df = best_upstream_ligands %>% lapply(get_weighted_ligand_target_links,geneset = target_genes[0:n], ligand_target_matrix = ligand_target_matrix, n = 250) %>% bind_rows()
    active_ligand_target_links_df = active_ligand_target_links_df %>% mutate(target_type = target_celltype) %>% inner_join(ligand_type_indication_df) 
    active_ligand_target_links_df = na.omit(active_ligand_target_links_df)
    

    p_ligand_target_network = ligand_target_heatmap(active_ligand_target_links_df, ligand_target_matrix, best_upstream_ligands)
    p_ligand_receptor_network = ligand_receptor_heatmap(lr_network, best_upstream_ligands, expressed_receptors, weighted_networks_lr)




    return(list(ligand_activities = ligand_activities, 
            p_ligand_target_network = p_ligand_target_network, 
            p_ligand_receptor_network = p_ligand_receptor_network))


}

In [16]:
table(seuratObj@meta.data$celltype_broad)


       Adipocytes Endothelial_Cells       Fibroblasts      Immune_Cells 
             6429               876              1077               577 
Mesothelial_Cells    Neuronal_Cells         Pericytes              SMCs 
              193                22               426                 8 

In [17]:
cell_types <- c(
  "Adipocytes",       
  "Endothelial_Cells",     
  "Fibroblasts",            
  "Immune_Cells",  
  "Mesothelial_Cells",  
  "Neuronal_Cells",      
  "Pericytes",   
  "SMCs" 
)

num <- length(cell_types)
print(paste("Number of cell types:", num))

[1] "Number of cell types: 8"


In [18]:
pb <- txtProgressBar(min = 0,      # Minimum value of the progress bar
                     max = length(cell_types)^2, # Assuming you iterate through each pair
                     style = 3,    # Progress bar style (also available style = 1 and style = 2)
                     width = 50,   # Progress bar width. Defaults to getOption("width")
                     char = "=")   # Character used to create the bar

i = 1
pvat_8weeks_hf_male_nichenet = list()

for (source_cell in cell_types) {
  for (target_cell in cell_types) {

      key = paste(gsub(" ", "", source_cell), "2", gsub(" ", "", target_cell), sep="")
      
      pvat_8weeks_hf_male_nichenet[[key]] <- tryCatch({
        ligand_receptor_activites(source_cell, target_cell, cell_data, nichenet_networks)
      }, error = function(e) {
        cat("Error with", source_cell, "to", target_cell, ":", e$message, "\n")
        NULL 
      })
    
    i = i + 1
    setTxtProgressBar(pb, i)
  }
}
close(pb)

  |                                                  |   0%

Joining with `by = join_by(ligand)`


  |==                                                |   3%

Joining with `by = join_by(ligand)`


  |==                                                |   5%

...

Joining with `by = join_by(ligand)`


  |================================================= |  98%

Joining with `by = join_by(ligand)`


  |==================================================| 100%

Joining with `by = join_by(ligand)`


In [20]:
names_interactions <- names(pvat_8weeks_hf_male_nichenet)
for (interaction_name in names_interactions) {
  interaction <- pvat_8weeks_hf_male_nichenet[[interaction_name]]
  
  matrix_names <- c("ligand_activities", "p_ligand_receptor_network", "p_ligand_target_network")
  for (matrix_name in matrix_names) {
    tryCatch({
      if (!is.null(interaction[[matrix_name]])) {
        interaction[[matrix_name]] <- as.data.frame(interaction[[matrix_name]])
      }
    }, error = function(e) {
       cat("Issue converting", matrix_name, "in", interaction_name, "to a data.frame:", e$message, "\n")
    })
  }
  pvat_8weeks_hf_male_nichenet[[interaction_name]] <- interaction
}

In [21]:
saveRDS(pvat_8weeks_hf_male_nichenet ,'pvat_8weeks_hf_male_nichenet.rds')

In [6]:
hf_male <- readRDS('pvat_8weeks_hf_male_nichenet.rds')


In [7]:
hf_male$Adipocytes2Endothelial_Cells$p_ligand_receptor_network

,Col4a2,Col7a1,Col5a3,Col5a1,Col27a1,Col11a2,Col12a1,Adam9,Lama4,Col4a1,⋯,Plxna2,Nrg2,Itgb1,Gpc6,Gnas,Glg1,Efna5,Cdh2,Adipoq,Apoo
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ephb1,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.5777388,0.0000000,0.0000000,0.0000000
Epha3,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,1.2313839,0.0000000,0.0000000,0.0000000
Epha4,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.9856822,0.0000000,0.0000000,0.0000000
Fzd6,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000
Lrp6,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000
Cd47,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000
Lrp5,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000
Notch1,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000
Notch4,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000
